## Matching up images with labels and cleaning dataset

**Import libraries**

In [ ]:
import pandas as pd
import os

In [ ]:
import numpy as np

In [ ]:
import time
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import cv2

**Import image reading movie set a pickle**

In [ ]:
movie = pd.read_pickle('/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/movie.pkl')

**Read the images as np arrays list for use in the model. Also keep a list of titles who's index matches the list of image np arrays**

In [ ]:
x = []
x_titles = []
for name in tqdm((os.listdir("/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/Images/"))):
    sb = "/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/Images/"+name
    s = name[:-4]
    x_titles.append(s)
    img = cv2.imread(sb)
    x.append(img)

**Find np arrays who's shapes do not match the standard shape: (268, 182, 3)**

In [ ]:
shapes_list = {}
for i in range(len(x)):
    if(x[i].shape not in shapes_list):
        shapes_list[x[i].shape] = 1
    else:
        shapes_list[x[i].shape] += 1

In [ ]:
shapes_list

The model needs all thei nputs to be the same size. The standard size in our dataset is (268, 182, 3), so we remove all data who's shape does not match.

In [ ]:
x[0].shape

In [ ]:
bad_shapes = []

In [ ]:
for i in range(len(x)):
    if (x[i].shape) != (x[0].shape):
        bad_shapes.append(i)
bad_shapes

** Delete the misshapen data from the titles list and image array list**

In [ ]:
counter = 0
for i in bad_shapes:
    del x[i-counter]
    del x_titles[i-counter]
    counter+= 1

In [ ]:
%matplotlib inline
plt.imshow(x[65])

** Create a new dataframe from the list of titles that has all the features of the csv **

This is done so that our image set's indexes match up with the dataframe's indexes. This is important in assigning genre labels to each image.

In [ ]:
titles = pd.DataFrame(x_titles, columns = ['Title'])

In [ ]:
df = pd.merge(titles, movie,how = 'inner', on='Title', copy = False)

In [ ]:
df = df.drop_duplicates(subset = 'Title')

In [ ]:
titles2 = df.Title.tolist()

**This deletes titles with odd characters as their title names are not able to be saved**

In [ ]:
excluded = []
excluded_titles = []
for i in range(len(x_titles)):
    if(x_titles[i] not in titles2):
        excluded.append(i)
        excluded_titles.append(x_titles[i])

In [ ]:
counter = 0
for i in excluded:
    del x[i-counter]
    del x_titles[i-counter]
    counter+= 1

**Check to make sure the lengths of the labels and images are the same**

In [ ]:
len(x)

In [ ]:
len(x_titles)

In [ ]:
plt.imshow(x[46])

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
len(df.Title)

In [ ]:
col_list = ['Title', 'Genre']

In [ ]:
df = df[col_list]

In [ ]:
df.head()

** Implement One-Hot encoding for the genre labels**

One-Hot encoding will save time for our model and also makes it much easier for the model to classify multi-label objects

In [ ]:
df2 = df['Genre'].str.join(sep='').str.get_dummies(sep='|')

In [ ]:
df2.head()

In [ ]:
df2 = df2.join(df, how = 'outer')

In [ ]:
df2 = df2.drop(['Genre'],axis = 1)

In [ ]:
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df2[cols]
df.head()

**Save the new cleaned movie dataframe as a pickle file**

In [ ]:
df.to_pickle("cleaned_movie_set.pkl")

** Tried to pickle the images, but was unable to as the pickle file exceeds 4.1 gbs and so will have to reload it in the main notebook. However the images are saved as a zip on Box **

In [ ]:
# x = np.asarray(x)

In [ ]:
# x.dump("cleaned_image_set.pkl")